In [1]:
from helper import *
from Simulation.ForwardKinematics import ForwardKinematics
from Simulation.DifferentialKinematics import DifferentialKinematics

from neura_dual_quaternions import DualQuaternion, Quaternion
from Simulation.LinearInterpolator import LinearInterpolator

import roboticstoolbox as rtb
import spatialmath as sm
import spatialgeometry as sg
import numpy as np
from swift import Swift

import time
np.set_printoptions(precision=6, suppress=True, linewidth=1200, threshold=np.inf)

In [2]:
x1 = DualQuaternion.fromQuatPos(Quaternion.fromAxisAngle(np.pi*0.5, np.array([0,1,0])), np.array([0.815,0,1.138]))
x2 = DualQuaternion.fromQuatPos(Quaternion.fromAxisAngle(np.pi*0.5, np.array([0,1,0])), np.array([0.815,0.2,1.138]))   

q0 = np.array([0,0.0,0,np.pi*0.5,0,0,0])
fk = ForwardKinematics()
traj = LinearInterpolator(x1, x2, 3)
dk = DifferentialKinematics()

J = fk.jacobian6(q0)
Jb = fk.jacobian_body(q0)
print(Jb)
Jo = Jb[:3]
Jp = Jb[3:]
#print(J6[:3])
#print(J6[3:])

axes = [True, True, True, False, False, False]

print(Jb[axes, :])
# U, S, Vh = np.linalg.svd(Jb@Jb.T)

# print(U)
# print(S)

U, S, Vh = np.linalg.svd(Jb)


print(U)
print(S)
#print(Vh)

u6 = np.array([0.63, 0., 0., 0., 0.775, 0.])
vd =  np.array([0, 0., 0., 0., 1, 0.])

res = np.dot(u6, vd)
print(res)


[[-1.     0.    -1.     0.     0.     0.     0.   ]
 [ 0.     1.     0.     1.     0.     1.     0.   ]
 [ 0.     0.     0.     0.     1.     0.     1.   ]
 [ 0.     0.815  0.     0.815  0.     0.115  0.   ]
 [ 0.815  0.     0.815  0.     0.     0.     0.   ]
 [ 0.     0.7    0.     0.     0.     0.     0.   ]]
[[-1.  0. -1.  0.  0.  0.  0.]
 [ 0.  1.  0.  1.  0.  1.  0.]
 [ 0.  0.  0.  0.  1.  0.  1.]]
[[ 0.        0.775165  0.        0.        0.        0.631759]
 [-0.819717  0.       -0.        0.491969  0.293309  0.      ]
 [-0.        0.       -1.       -0.        0.        0.      ]
 [-0.525233  0.        0.       -0.44139  -0.727533  0.      ]
 [ 0.       -0.631759  0.        0.        0.        0.775165]
 [-0.22846   0.        0.       -0.750427  0.620213  0.      ]]
[2.076825 1.824404 1.414214 0.605517 0.389646 0.      ]
0.775


In [3]:
q1 = np.array([-0.677,  0.208,  0.791,  1.547,  0.388, -0.141, -0.531])
qd = np.ones(7)
print(qd)

H = fk.hessian(q1)
J = fk.jacobian6(q1)

print(H.shape)

W = np.diag([1,1,1,1,1,1])
manipulability = np.linalg.det(J@J.T)

print(J@J.T)
b = np.linalg.pinv(J@J.T)
Jm = np.zeros((7, 1))

for i in range(7):
    #print(H[:,i,:].T)
    c = W@J@H[:,:,i].T
    #print(c)
    Jm[i,0] = manipulability*(c.flatten().T)@b.flatten()
    
print(Jm)
print(dk.manipulability_gradient(q1))



[1. 1. 1. 1. 1. 1. 1.]
(6, 7, 7)
[[ 2.398773  0.498709  0.000278 -0.226345  2.354829  0.107815]
 [ 0.498709  2.350096  0.43987  -2.162534 -0.12623   0.708818]
 [ 0.000278  0.43987   2.251132 -0.548341 -0.316257  0.352575]
 [-0.226345 -2.162534 -0.548341  2.191454  0.268836 -0.747534]
 [ 2.354829 -0.12623  -0.316257  0.268836  2.630473 -0.161609]
 [ 0.107815  0.708818  0.352575 -0.747534 -0.161609  0.499574]]
[[ 0.      ]
 [ 0.065284]
 [ 0.00122 ]
 [-0.041684]
 [-0.000729]
 [-0.02374 ]
 [-0.001047]]
[-0.        0.35944  -0.025166  0.005984 -0.031129 -0.170192 -0.      ]


In [4]:
q1 = np.array([-0.677,  0.208,  0.791,  1.547,  0.388, -0.141, -0.531])
q2 = np.array([-0.687,  0.21,   0.804,  1.547,  0.393, -0.142, -0.54])
q3 = (q1 + q2)*0.5
v = [1, 0, 0, 0, 1, 0]
print(dk.dir_manipulability(q1, v))
print(dk.dir_manipulability(q3, v))
print(dk.dir_manipulability(q2, v))

print(dk.dir_manipulability_gradient(q1, v))
print(dk.dir_manipulability_gradient(q3, v))
print(dk.dir_manipulability_gradient(q2, v))

Jb = fk.jacobian_body(q1)
U, S, Vh = np.linalg.svd(Jb@Jb.T)
print(U)
print(S)

Jb = fk.jacobian_body(q3)
U, S, Vh = np.linalg.svd(Jb@Jb.T)
print(U)
print(S)

Jb = fk.jacobian_body(q2)
U, S, Vh = np.linalg.svd(Jb@Jb.T)
print(U)
print(S)

1.6903383455160006
1.5173263542385491
1.5821155398748201
[ 0.       -6.668876  7.50717   4.017953 -2.83828   3.544385 -1.889526]
[  0.       -86.416601 -96.089278 -47.177078 -95.924938   4.83087   -1.779227]
[ -0.        30.92397   -1.49334   -1.994111  24.239465 -10.270669  -1.709053]
[[ 0.002858  0.767771  0.014095 -0.061606  0.415122  0.483942]
 [ 0.692729 -0.157546  0.404186  0.500268  0.014395  0.285419]
 [ 0.470483 -0.001957 -0.875844  0.070411  0.075415 -0.029891]
 [ 0.492441 -0.033193  0.241136 -0.726192  0.287896 -0.296671]
 [-0.196489 -0.619898 -0.096665 -0.177246  0.473892  0.558376]
 [ 0.13288   0.018102 -0.042904 -0.426858 -0.717162  0.532584]]
[3.767832 3.756263 2.100892 0.226468 0.190467 0.008089]
[[ 0.491415  0.590226 -0.014414  0.072163  0.415946  0.481371]
 [ 0.431243 -0.561918 -0.407549 -0.499077  0.028328  0.286882]
 [ 0.364733 -0.304235  0.87341  -0.068311  0.077504 -0.029887]
 [ 0.357298 -0.338784 -0.243833  0.7332    0.267819 -0.297993]
 [-0.546809 -0.351587  0.0

In [ ]:
env = Swift()
robot = rtb.models.URDF.Maira7M()

env.launch(realtime=True, browser="notebook")
env.add(robot)

# end-effector axes
ee_axes = sg.Axes(0.1)

env.add(ee_axes)
np.set_printoptions(suppress=True, precision=3)

# Change the robot configuration to the ready position
robot.q = np.array([0, 0.0, 0, np.pi / 2, 0.00, -0.0, 0])
robot.qd = np.zeros(robot.n)




# Step the sim to view the robot in this configuration
env.step(0)

# Specify our desired end-effector velocity
ev = [0, 0.0, 0.0, 0, 0.0, 0]
ev2 = [1, 0.0, 0.0, 0.0, 0.0, 0.0]
# Specify our timestep
dt = 0.005


# Run the simulation for 4 seconds
for i in range(1000):

    # Work out the manipulator Jacobian, Hessian using the
    # current robot configuration
    J = fk.jacobian_body(robot.q)
    grad = dk.manipulability_gradient(robot.q)
    grad2 = dk.dir_manipulability_gradient(robot.q, ev2)
    print(dk.manipulability(robot.q), dk.dir_manipulability(robot.q, ev2))
    # Calculate δq, the change to qd
    pinv = np.linalg.pinv(J)
    v = pinv @ ev + 20*(np.eye(7) - pinv@J)@grad2

    #robot.qd = qd - δq
    robot.q += v*dt
    
    # Update the ee axes
    ee_axes.T = fk.forward_kinematics(robot.q).asTransformation()

    # Step the simulator by dt seconds
    env.step(dt)